# Dataset Sample Visualization

This notebook demonstrates how to download the mm-whs, apis, or CHAOS datasets on Kaggle, load them with MONAI transforms, and visualize a single volumetric sample.

In [ ]:
!pip install -q monai matplotlib kaggle

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import torch
from monai.transforms import Compose, LoadImaged, AddChanneld, ScaleIntensityd, EnsureTyped
from monai.data import CacheDataset, DataLoader
from monai_project.interface import visualize_sample

In [ ]:
# Select dataset: one of 'CHAOS', 'MMWHS', 'APIS'
DATASET = 'CHAOS'
if DATASET == 'CHAOS':
    !kaggle datasets download -q thewilliamraisely/chaos-mri -f chaos_mri.zip
    !unzip -q chaos_mri.zip -d chaos
    data_dir = 'chaos'
elif DATASET == 'MMWHS':
    # Install Mega client for Python
    !pip install -q megapy
    from mega import Mega
    mega = Mega()
    # Replace with actual Mega file URL for MMWHS
    mmwhs_url = 'https://mega.nz/folder/UNMF2YYI#1cqJVzo4p_wESv9P_pc8uA'
    mega.download_url(mmwhs_url, dest_filename='mm_whs.zip')
    !unzip -q mm_whs.zip -d mmwhs
    data_dir = 'mmwhs'
elif DATASET == 'APIS':
    !kaggle datasets download -q thewilliamraisely/apis -f apis.zip
    !unzip -q apis.zip -d apis
    data_dir = 'apis'
else:
    raise ValueError(f"Unknown dataset: {DATASET}")
print(f"Data directory: {data_dir}")

In [ ]:
# Prepare data dictionaries for image and label files
images = sorted(glob.glob(os.path.join(data_dir, 'images', '*.nii.gz')))
labels = sorted(glob.glob(os.path.join(data_dir, 'labels', '*.nii.gz')))
data_dicts = [{'image': img, 'label': lbl} for img, lbl in zip(images, labels)]

# Define MONAI transforms for loading and processing
transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    AddChanneld(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    EnsureTyped(keys=['image', 'label']),
])

# Create dataset and DataLoader
dataset = CacheDataset(data=data_dicts, transform=transforms)
dataloader = DataLoader(dataset, batch_size=1)

In [ ]:
# Visualize the first sample from the DataLoader
visualize_sample(dataloader, sample_index=0)